Objetivo: Cortar y rasterizar el shapefile de manglares de México para entrenar un clasificador de cobertura para imágenes landsat. 

In [1]:
import landsat.search as search
import landsat.downloader as downloader
from landsat.image import Simple, PanSharpen, FileDoesNotExist
import rasterio
import subprocess
import numpy
import glob
import os
import shutil

In [4]:
scenes = search.Search().search('031,045', limit = 1, cloud_max = 3, start_date = "2012", end_date = "2016")

In [5]:
scenes_ids = [str(scene['sceneID']) for scene in scenes['results']]
print scenes_ids

['LC80310452015085LGN00']


In [6]:
download = downloader.Downloader()

In [8]:
for s_id in scenes_ids:
    output = download.download([s_id])

    17%     94.3 MiB     555 Bytes/s   10 days, 0:17:33 ETA    
    17%     95.4 MiB     727.7 KiB/s            0:10:43 ETA    
    17%     96.9 MiB       1.3 MiB/s            0:05:46 ETA    
    17%     98.7 MiB       1.7 MiB/s            0:04:23 ETA    
    18%    100.5 MiB       2.0 MiB/s            0:03:42 ETA    
    18%    102.0 MiB       2.2 MiB/s            0:03:27 ETA    
    18%    103.6 MiB       2.3 MiB/s            0:03:13 ETA    
    19%    105.3 MiB       2.4 MiB/s            0:03:04 ETA    
    19%    107.1 MiB       2.6 MiB/s            0:02:54 ETA    
    19%    108.9 MiB       2.6 MiB/s            0:02:48 ETA    
    20%    110.6 MiB       2.7 MiB/s            0:02:43 ETA    
    20%    111.9 MiB       2.7 MiB/s            0:02:44 ETA    
    20%    113.5 MiB       2.7 MiB/s            0:02:41 ETA    
    20%    115.2 MiB       2.8 MiB/s            0:02:38 ETA    
    21%    116.4 MiB       2.7 MiB/s            0:02:39 ETA    
    21%    117.9 MiB       2.8 MiB/s    

===> Downloading: LC80310452015085LGN00.tar.bz
     stored at /Users/RaulSierra/landsat/downloads


In [9]:
landsat_dir = "/Users/RaulSierra/landsat/downloads/"
landsat_files = glob.glob(landsat_dir + "LC*.tar.bz")

for file in landsat_files[:2]:
    im_process = Simple(file, bands=[4,3,2,1,5,6,7,9], dst_path="Darwin-SFL-101/landsat_data/", verbose=False)
    im_process.run()

===> Unzipping LC80310452015085LGN00 - It might take some time
===> Image processing started for bands 4-3-2-1-5-6-7-9
===> Getting boundaries
===> Projecting
     band 4
     band 3
     band 2
     band 1
     band 5
     band 6
     band 7
     band 9
===> Calculating cloud and snow coverage from QA band
     cloud/snow coverage: 1.87
===> Final Steps
     Color correcting band 4
     Color correcting band 3
     Color correcting band 2
     Color correcting band 1
     Color correcting band 5
     Color correcting band 6
     Color correcting band 7
     Color correcting band 9
     Writing to file


In [ ]:
landsat_data_dir = "Darwin-SFL-101/landsat_data/"
for (root, dirs, files) in os.walk(landsat_data_dir):
    for file in files:
        if file.endswith(".TIF"):
            shutil.move(os.path.join(root,file), landsat_data_dir)
            
    if not dirs:
        shutil.rmtree(root)

## Crear raster de entrenamiento

In [ ]:
!mapshaper Darwin-SFL-101/mapa_manglar/mexoc2010gw/mexoc2010gw.shp -filter-fields Clase -o manglar_clase.shp

Ahora hay que recortar el shape a solo la escena que vamos a usar

In [ ]:
bbox = "-107.31526908421758,20.59854358911711,-105.07951516054297,22.730735318220372"
!mapshaper manglar_clase.shp -clip bbox=$bbox -o manglar_clase_clip.shp 

Utilizamos rasterio para crear un raster en ceros donde rasterizemosel shapefile

In [ ]:
landsat_scene = "/Users/RaulSierra/Ecoinformatica/Devel/iPyNotebooks/Darwin/Darwin-SFL-101/train/landsat/LC80310452014130LGN00_bands_4321567891011.tif"
gt_raster = landsat_scene.replace(".tif", ".gt")

with rasterio.drivers():
    # Read raster bands directly to Numpy arrays.
    #
    with rasterio.open(landsat_scene) as src:
        width = src.width
        height = src.height
        empty_rast = numpy.zeros((width, height), dtype = numpy.uint8)

        
        # Write the product as a raster band to a new 8-bit file. For
        # the new file's profile, we start with the meta attributes of
        # the source file, but then change the band count to 1, set the
        # dtype to uint8, and specify LZW compression.
        profile = src.profile
        profile.update(
            dtype=rasterio.uint8,
            count=1,
            compress='lzw',
            photometric = 'MINISBLACK')

        with rasterio.open(gt_raster, 'w', **profile) as dst:
            dst.write(empty_rast.astype(rasterio.uint8), 1)

    # At the end of the ``with rasterio.drivers()`` block, context
    # manager exits and all drivers are de-registered.

In [ ]:
!rio info $landsat_scene

In [ ]:
!ogr2ogr -t_srs EPSG:3857 manglar_clase_clip_new.shp manglar_clase_clip.shp
!gdal_rasterize -a Clase -l manglar_clase_clip_new manglar_clase_clip_new.shp $gt_raster

In [ ]:
!gdal_calc.py -A $gt_raster -B $landsat_scene --outfile=$gt_raster --calc="A*(B > 0)"

Copiamos el groundtruth para todas las imágenes

In [176]:
ls_files = glob.glob(landsat_data_dir + "*.TIF")
for ls_file in ls_files:
    shutil.copy(gt_raster, ls_file.replace(".TIF", ".gt"))

Limpiar set de entrenamiento

In [177]:
tiles_dir = "./Darwin-SFL-101/train/"
gt_tiles = glob.glob(tiles_dir + "*.gt")

with rasterio.drivers():
    # Read raster bands directly to Numpy arrays.
    #
    for tile in gt_tiles:
        with rasterio.open(tile) as src:
            data = src.read()
            if(data.sum() == 0):
                os.remove(tile)
                os.remove(tile.replace(".gt", ".tif"))                
 

In [ ]:
rapideye_name = "1349112_2014-05-17_RE4_3A_255761"
!rio clip $landsat_scene {"LS_" + rapideye_name  + ".tif"} --like {train_dir + rapideye_name + ".tif"}

In [ ]:
!rio warp {train_dir + rapideye_name + ".gt"} {train_dir + "landsat/LS_" + rapideye_name  + ".gt"} --like {train_dir + "landsat/LS_" + rapideye_name  + ".tif"}

In [ ]:
!rio info {train_dir + "landsat/LS_" + rapideye_name  + ".tif"}